# 01: Neural Network Regression with TensorFlow

As we all know that, there are many pre-defined definitions for a [`regression_problem`](https://en.wikipedia.org/wiki/Regression_analysis) but in our case, to simplify it to be: predicting a number:

For example, you might want to:
* Estimate or predict the cost of medical insurance based on a person's demographics (age, gender, sex, race).
* Estimate or predict the price of a property based on the information you have about it (such as number of rooms, number of bathrooms)
* Estimate or predict the co-ordinate of a bounding box of an item in an image.

In this notebook, we are going to set the foundations for how can you build a neural network to detect patterns in a sample of inputs (this is your data), and then making a estimation/prediction (in a form of a number) based on those inputs.

## What we are going to cover?

Specially, we will go through how to use `TensorFlow` to perform the following:

* Architecture of the regression model
* Input shapes and output shapes
    * `X`: Feature / Data (inputs)
    * 'y': Label / Target (outputs)
* Creating custom data to view and fit
* Steps in modelling 
    * Creating a model
    * Compiling a model
        * Defining a loss function
        * Setting up an optimizer 
        * Creating an evaluation metrics
    * Fitting a model (getting it to find patterns in your data)
* Evaluating a model
    * Visualizing the model (`"(3V) visualize, visualize, and visualize"`)
    * Looking at training curves
    * Compare estimations/predictions to ground truth (using our evaluation metrices)
* Saving a model (So we can use it for later)
* Loading a saved model

> 🔑 **Note:** Don't worry if none of these make sense right now, we'll go through each of them one by one.

## How you can use this notebook?

You can go through the explanations and code (everything should work), but there's a better way to run code.

Write all of the code yourself.

Yes. I'm serious. Create a new notebook, and rewrite each line by yourself. Investigate it, see if you can break it, why does it break?

You don't have to write the text descriptions but writing the code yourself is a great way to get hands-on experience.

Don't worry if you make mistakes, we all do. The way to get better and make less mistakes is to **write more code**.

## Typical architecture of a regression neural network

The word `typical` is on purpose. 

Why?

Because there are so many distict methods to build a neural networks (almost an unlimited number of ways).

However, the following is a typical framework for taking a set of numbers, detecting patterns in them, and then getting a target number.

Yes, we know the previous sentence is vague but we'll see this in action shortly.

| **Hyperparameter** | **Typical value** |
| --- | --- |
| Input layer shape | Same shape as number of features (e.g. 3 for # bedrooms, # bathrooms, # car spaces in housing price prediction) |
| Hidden layer(s) | Problem specific, minimum = 1, maximum = unlimited |
| Neurons per hidden layer | Problem specific, generally 10 to 100 |
| Output layer shape | Same shape as desired prediction shape (e.g. 1 for house price) |
| Hidden activation | Usually [ReLU](https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning) (rectified linear unit) |
| Output activation | None, ReLU, logistic/tanh |
| Loss function | [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) (mean square error) or [MAE](https://en.wikipedia.org/wiki/Mean_absolute_error) (mean absolute error)/Huber (combination of MAE/MSE) if outliers |
| Optimizer | [SGD](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD) (stochastic gradient descent), [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) |

***Table 1:*** *Typical architecture of a regression network.* ***Source:*** *Adapted from page 293 of [Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow Book by Aurélien Géron](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)*

Again, if you're new to neural networks and deep learning in general, much of the above table won't make sense. But don't worry, we'll be getting hands-on with all of it soon.

> 🔑 **Note:** A **hyperparameter** in machine learning is something a data analyst or developer can set themselves, where as a **parameter** usually describes something a model learns on its own (a value not explicitly set by an analyst).

Okay, enough talk, let's get started writing code.

To use TensorFlow, we'll import it as the common alias `tf` (short for TensorFlow).